<a href="https://colab.research.google.com/github/mtermor/NTIC_DeepLearning/blob/main/RNN/Tarea_RNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Dado que el entrenamiento de redes neuronales es una tarea  muy costosa, **se recomienda ejecutar el notebooks en [Google Colab](https://colab.research.google.com)**, por supuesto también se puede ejecutar en local.

Al entrar en [Google Colab](https://colab.research.google.com) bastará con hacer click en `upload` y subir este notebook. No olvide luego descargarlo en `File->Download .ipynb`

**El examen deberá ser entregado con las celdas ejecutadas, si alguna celda no está ejecutadas no se contará.**

El examen se divide en tres partes, con la puntuación que se indica a continuación. La puntuación máxima será 10.

    
- [Actividad 1: Redes Recurrentes](#actividad_1): 10 pts
    - [Cuestión 1](#3.1): 2.5 pt
    - [Cuestión 2](#3.2): 2.5 pt
    - [Cuestión 3](#3.3): 2.5 pts
    - [Cuestión 4](#3.4): 1.25 pts
    - [Cuestión 5](#3.5): 1.25 pts



In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

<a name='actividad_1'></a>
# Actividad 1: Redes Recurrentes


- [Cuestión 1](#3.1): 2.5 pt
- [Cuestión 2](#3.2): 2.5 pt
- [Cuestión 3](#3.3): 2.5 pts
- [Cuestión 4](#3.4): 1.25 pts
- [Cuestión 5](#3.5): 1.25 pts

Vamos a usar un dataset de las temperaturas mínimas diarias en Melbourne. La tarea será la de predecir la temperatura mínima en dos días. Puedes usar técnicas de series temporales vistas en otras asignaturas, pero no es necesario.


In [2]:
dataset_url = 'https://raw.githubusercontent.com/jbrownlee/Datasets/master/daily-min-temperatures.csv'
data_dir = tf.keras.utils.get_file('daily-min-temperatures.csv', origin=dataset_url)

67921/67921 [==============================] - 0s 0us/step


In [5]:
df = pd.read_csv(data_dir, parse_dates=['Date'])
df.head()

,Date,Temp
0,1981-01-01,20.7
1,1981-01-02,17.9
2,1981-01-03,18.8
3,1981-01-04,14.6
4,1981-01-05,15.8


In [7]:
temperatures = df['Temp'].values
print('number of samples:', len(temperatures))
train_data = temperatures[:3000]
test_data = temperatures[3000:]
print('number of train samples:', len(train_data))
print('number of test samples:', len(test_data))
print('firsts train samples:', train_data[:10])

number of samples: 3650
number of train samples: 3000
number of test samples: 650
firsts train samples: [20.7 17.9 18.8 14.6 15.8 15.8 15.8 17.4 21.8 20. ]


<a name='3.1'></a>
## Cuestión 1: Convierta `train_data` y `test_data`  en ventanas de tamaño 5, para predecir el valor en 2 días

En la nomenclatura de [Introduction_to_RNN_Time_Series.ipynb](https://github.com/ezponda/intro_deep_learning/blob/main/class/RNN/Introduction_to_RNN_Time_Series.ipynb)
```python
past, future = (5, 2)
```

Para las primeras 10 muestras de train_data `[20.7, 17.9, 18.8, 14.6, 15.8, 15.8, 15.8, 17.4, 21.8, 20. ]` el resultado debería ser:

```python
x[0] : [20.7, 17.9, 18.8, 14.6, 15.8] , y[0]: 15.8
x[1] : [17.9, 18.8, 14.6, 15.8, 15.8] , y[1]: 17.4
x[2] : [18.8, 14.6, 15.8, 15.8, 15.8] , y[2]: 21.8
x[3] : [14.6, 15.8, 15.8, 15.8, 17.4] , y[3]: 20.             
```

In [8]:
# Windowing function
def create_windows_np(data, window_size, horizon, shuffle=False):
    """
    Creates a dataset from the given time series data using NumPy.

    Parameters:
    data (np.ndarray): Time series data with one dimension.
    window_size (int): The number of past time steps to use as input features.
    horizon (int): The number of future time steps to predict.
    shuffle (bool): Shuffle the windows or not.

    Returns:
    tuple: A tuple containing the input-output pairs (windows, targets) as NumPy arrays.
    """

    X, y = [], []
    for i in range(len(data) - window_size - horizon + 1):
        X.append(data[i:i+window_size])
        y.append(data[i+window_size+horizon-1])

    X, y = np.array(X), np.array(y)

    if shuffle:
        indices = np.arange(len(X))
        np.random.shuffle(indices)
        X, y = X[indices], y[indices]

    return X, y

In [10]:
# Comprobar el resultado para las primeras 10 muestras
Xt, yt = create_windows_np([20.7, 17.9, 18.8, 14.6, 15.8, 15.8, 15.8, 17.4, 21.8, 20. ],
                           window_size = 5,
                           horizon = 2,
                           shuffle=False)
for ind in range(len(yt)):
    print(Xt[ind, :], yt[ind])

[20.7 17.9 18.8 14.6 15.8] 15.8
[17.9 18.8 14.6 15.8 15.8] 17.4
[18.8 14.6 15.8 15.8 15.8] 21.8
[14.6 15.8 15.8 15.8 17.4] 20.0


In [14]:
past, future = (5, 2)
X_train, y_train = create_windows_np(train_data,
                           window_size = past,
                           horizon = future,
                           shuffle=False)
X_test, y_test = create_windows_np(test_data,
                           window_size = past,
                           horizon = future,
                           shuffle=False)

<a name='3.2'></a>
## Cuestión 2: Cree un modelo recurrente de dos capas GRU para predecir con las ventanas de la cuestión anterior.


In [15]:
inputs = keras.layers.Input(shape=(past, 1))

GRU_1 = keras.layers.GRU(64, return_sequences=True)(inputs)
GRU_2 = keras.layers.GRU(64, return_sequences=False)(GRU_1)

outputs = layers.Dense(1)(GRU_2)

model = keras.Model(inputs=inputs, outputs=outputs)
model.compile(optimizer=keras.optimizers.Adam(), loss='mse')
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 5, 1)]            0         
                                                                 
 gru (GRU)                   (None, 5, 64)             12864     
                                                                 
 gru_1 (GRU)                 (None, 64)                24960     
                                                                 
 dense (Dense)               (None, 1)                 65        
                                                                 
Total params: 37889 (148.00 KB)
Trainable params: 37889 (148.00 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [16]:
es_callback = keras.callbacks.EarlyStopping(
    monitor="val_loss", min_delta=0, patience=10)

history = model.fit(
    X_train, y_train,
    epochs=200,
    validation_split=0.2, shuffle=True, batch_size = 64, callbacks=[es_callback]
)

Epoch 1/200
38/38 [==============================] - 6s 24ms/step - loss: 40.4479 - val_loss: 16.8368
Epoch 2/200
38/38 [==============================] - 0s 6ms/step - loss: 16.0726 - val_loss: 16.1170
Epoch 3/200
38/38 [==============================] - 0s 6ms/step - loss: 13.8299 - val_loss: 13.1819
Epoch 4/200
38/38 [==============================] - 0s 6ms/step - loss: 11.2317 - val_loss: 10.5913
Epoch 5/200
38/38 [==============================] - 0s 7ms/step - loss: 9.7725 - val_loss: 9.2979
Epoch 6/200
38/38 [==============================] - 0s 7ms/step - loss: 9.2637 - val_loss: 8.9224
Epoch 7/200
38/38 [==============================] - 0s 6ms/step - loss: 9.0427 - val_loss: 8.8124
Epoch 8/200
38/38 [==============================] - 0s 7ms/step - loss: 8.9682 - val_loss: 8.5113
Epoch 9/200
38/38 [==============================] - 0s 6ms/step - loss: 8.9992 - val_loss: 8.8112
Epoch 10/200
38/38 [==============================] - 0s 7ms/step - loss: 8.8390 - val_loss: 8.6773


In [20]:
results = model.evaluate(X_test, y_test, verbose=1)
print('Test Loss: {}'.format(results))

21/21 [==============================] - 0s 4ms/step - loss: 6.9687
Test Loss: 6.968696117401123


<a name='3.3'></a>
## Cuestión 3: Añada más features a la series temporal, por ejemplo `portion_year`. Cree un modelo que mejore al anterior.


In [18]:
## Puede añadir más features
df['portion_year'] = df['Date'].dt.dayofyear / 365.0
df_multi = df[['Temp', 'portion_year']].copy()

## train - test split
train_data = df_multi.iloc[:3000].copy()
test_data = df_multi.loc[3000:, :].copy()

In [30]:
def create_windows_multivariate_np(data, window_size, horizon, target_col_idx, shuffle=False):
    """
    Creates a dataset from the given time series data using NumPy.

    Parameters:
    data (np.ndarray or pd.DataFrame): Time series data with multiple features.
    window_size (int): The number of past time steps to use as input features.
    horizon (int): The number of future time steps to predict.
    target_col_idx (int): The index of the target column in the input data.
    shuffle (bool): Whether to shuffle the data or not.

    Returns:
    tuple: A tuple containing the input-output pairs (X, y) as NumPy arrays.
    """
    if isinstance(data, pd.DataFrame):
        data = data.values

    X, y = [], []
    for i in range(len(data) - window_size - horizon + 1):
        X.append(data[i:i+window_size, :])
        y.append(data[i+window_size+horizon-1, target_col_idx])

    X, y = np.array(X), np.array(y)

    if shuffle:
        indices = np.arange(X.shape[0])
        np.random.shuffle(indices)
        X, y = X[indices], y[indices]

    return X, y


In [31]:
## Create windows
X_train, y_train = create_windows_multivariate_np(
    train_data, past, future, target_col_idx=0, shuffle=True)

X_test, y_test = create_windows_multivariate_np(
    test_data, past, future, target_col_idx=0, shuffle=True)

In [37]:
inputs = keras.layers.Input(shape=(past, X_train.shape[2]))

GRU_1 = keras.layers.GRU(64, return_sequences=True)(inputs)
GRU_2 = keras.layers.GRU(64, return_sequences=False)(GRU_1)

outputs = layers.Dense(1)(GRU_2)

model = keras.Model(inputs=inputs, outputs=outputs)
model.compile(optimizer=keras.optimizers.Adam(), loss='mse')
model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 5, 2)]            0         
                                                                 
 gru_2 (GRU)                 (None, 5, 64)             13056     
                                                                 
 gru_3 (GRU)                 (None, 64)                24960     
                                                                 
 dense_1 (Dense)             (None, 1)                 65        
                                                                 
Total params: 38081 (148.75 KB)
Trainable params: 38081 (148.75 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [38]:
es_callback = keras.callbacks.EarlyStopping(
    monitor="val_loss", min_delta=0, patience=10)

history = model.fit(
    X_train, y_train,
    epochs=200,
    validation_split=0.2, shuffle=True, batch_size = 64, callbacks=[es_callback]
)

Epoch 1/200
38/38 [==============================] - 4s 24ms/step - loss: 39.3963 - val_loss: 16.9322
Epoch 2/200
38/38 [==============================] - 0s 6ms/step - loss: 16.1141 - val_loss: 15.8754
Epoch 3/200
38/38 [==============================] - 0s 7ms/step - loss: 14.0486 - val_loss: 12.8490
Epoch 4/200
38/38 [==============================] - 0s 6ms/step - loss: 11.5386 - val_loss: 10.8356
Epoch 5/200
38/38 [==============================] - 0s 6ms/step - loss: 9.9917 - val_loss: 9.9499
Epoch 6/200
38/38 [==============================] - 0s 6ms/step - loss: 9.2894 - val_loss: 9.3088
Epoch 7/200
38/38 [==============================] - 0s 6ms/step - loss: 8.9864 - val_loss: 9.1115
Epoch 8/200
38/38 [==============================] - 0s 7ms/step - loss: 8.9525 - val_loss: 8.9921
Epoch 9/200
38/38 [==============================] - 0s 6ms/step - loss: 8.7764 - val_loss: 8.8934
Epoch 10/200
38/38 [==============================] - 0s 6ms/step - loss: 8.7386 - val_loss: 8.9243


In [39]:
results = model.evaluate(X_test, y_test, verbose=1)
print('Test Loss: {}'.format(results))

21/21 [==============================] - 0s 3ms/step - loss: 6.4087
Test Loss: 6.408729553222656


In [40]:
# Test loss con un single feature = 6.9687
# Test loss con multiples features = 6.4087

# Dado que el test loss es menor con múltiples features, el nuevo modelo (con
# múltiples características) ha mejorado en comparación con el modelo anterior
# (con una sola característica).


<a name='3.4'></a>
## Cuestión 4: ¿En cuáles de estas aplicaciones se usaría un arquitectura 'many-to-one'?

**a)** Clasificación de sentimiento en textos

**b)** Verificación de voz para iniciar el ordenador.

**c)** Generación de música.

**d)** Un clasificador que clasifique piezas de música según su autor.


La arquitectura "many-to-one" procesa una secuencia de datos y produce una única predicción o clasificación. Esta arquitectura se usaría en las siguientes aplicaciones:

* **a) Clasificación de sentimientos en textos:** se procesa una secuencia de palabras (un texto) para producir una única salida que representa el sentimiento (por ejemplo, positivo o negativo). Es importante notar que si existiesen más categorías, el modelo no sería adecuado ya que no podría clasificar un elemento en varias categorías a la vez.
* **b) Verificación de voz para iniciar el ordenador:** se analiza una secuencia de características de audio para verificar si la voz corresponde a una persona autorizada, produciendo una única salida de verificación.
* **d) Un clasificador que clasifique piezas de música según su autor:** se analiza una secuencia de características de una pieza musical para producir una única salida que indique el autor.

Sin embargo, la generación de música implica producir una secuencia de notas o sonidos, lo cual requiere una arquitectura 'many-to-many', donde tanto la entrada como la salida son secuencias.

<a name='3.5'></a>
## Cuestión 5: ¿Qué ventajas aporta el uso de word embeddings?

**a)** Permiten reducir la dimensión de entrada respecto al one-hot encoding.

**b)** Permiten descubrir la similaridad entre palabras de manera más intuitiva que con one-hot encoding.

**c)** Son una manera de realizar transfer learning en nlp.

**d)** Permiten visualizar las relaciones entre palabras con métodos de reducción de dimensioones como el PCA.


El uso de word embeddings aporta:
* **a) Permiten reducir la dimensión de entrada respecto al one-hot encoding:** A diferencia del one-hot encoding, que representa cada palabra como un vector con una dimensión por cada palabra posible en el vocabulario, las word embeddings representan cada palabra como un vector con una dimensión mucho menor.

* **b) Permiten descubrir la similaridad entre palabras de manera más intuitiva que con one-hot encoding:** Las word embeddings capturan las relaciones semánticas entre palabras, a través de representaciones vectoriales cercanas entre palabras con significados similares.

* **c) Son una manera de realizar transfer learning en nlp:**  Las word embeddings entrenadas en un conjunto de datos grande se pueden utilizar para inicializar un modelo de aprendizaje automático para una tarea diferente.

* **d) Permiten visualizar las relaciones entre palabras con métodos de reducción de dimensioones como el PCA:** Las word embeddings se pueden visualizar utilizando métodos de reducción de dimensionalidad como PCA, lo cual puede ser útil para comprender cómo el modelo de aprendizaje automático representa las relaciones semánticas entre palabras.





